In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image

In [2]:
# Function to load images and labels from folders names
def load_data(data_dir, img_width, img_height):
    images = []
    labels = []
    label_mapping = {'female': 0, 'male': 1}
    for folder in os.listdir(data_dir):
        if folder.startswith('.'):  # Skip hidden folders
            continue
        folder_path = os.path.join(data_dir, folder)
        label = label_mapping[folder.lower()]  # Get the integer label
        for file in os.listdir(folder_path):
            if file.endswith('.jpg'):
                image_path = os.path.join(folder_path, file)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (img_width, img_height))  # Resize images
                images.append(image)
                labels.append(label)
    return np.array(images), np.array(labels)

In [3]:
# Load and preprocess the dataset
path = '/content/drive/MyDrive/Colab Notebooks/gender.v1i.folder/train'
images, labels = load_data(path,128,128)
labels = labels.astype(np.int32)

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
175/175 [==============================] - 12s 19ms/step - loss: 6.9056 - accuracy: 0.7877 - val_loss: 0.2657 - val_accuracy: 0.8921
Epoch 2/10
175/175 [==============================] - 3s 15ms/step - loss: 0.2191 - accuracy: 0.9130 - val_loss: 0.2996 - val_accuracy: 0.8636
Epoch 3/10
175/175 [==============================] - 3s 16ms/step - loss: 0.1747 - accuracy: 0.9323 - val_loss: 0.2425 - val_accuracy: 0.9064
Epoch 4/10
175/175 [==============================] - 3s 17ms/step - loss: 0.1344 - accuracy: 0.9477 - val_loss: 0.2305 - val_accuracy: 0.9171
Epoch 5/10
175/175 [==============================] - 3s 17ms/step - loss: 0.0815 - accuracy: 0.9696 - val_loss: 0.2689 - val_accuracy: 0.9107
Epoch 6/10
175/175 [==============================] - 3s 16ms/step - loss: 0.0435 - accuracy: 0.9823 - val_loss: 0.2755 - val_accuracy: 0.9271
Epoch 7/10
175/175 [==============================] - 3s 16ms/step - loss: 0.0349 - accuracy: 0.9886 - val_loss: 0.4221 - val_accuracy: 0.892

In [7]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

44/44 [==============================] - 0s 6ms/step - loss: 0.3882 - accuracy: 0.9100
Test Loss: 0.38815367221832275
Test Accuracy: 0.9100000262260437


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
model.save('my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Function to preprocess user input image
def preprocess_image(image_path, target_size):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to make prediction using the loaded model
def predict_gender(image_array, model):
    # Perform prediction
    prediction = model.predict(image_array)
    # Map prediction to class label
    if prediction[0] > 0.5:
        gender = 'Male'
    else:
        gender = 'Female'
    return gender

In [11]:
# Preprocess user input image
image_path = '/content/drive/MyDrive/Colab Notebooks/gender.v1i.folder/test/Male/177404_jpg.rf.ab8423701037c25bed57ec81844b58c9.jpg'
input_size = (128, 128)  # Adjust this size according to your model's input size
processed_image = preprocess_image(image_path, input_size)

# Perform prediction
predicted_gender = predict_gender(processed_image, model)
print("Predicted gender:", predicted_gender)

1/1 [==============================] - 0s 386ms/step
Predicted gender: Male
